### Imports

In [86]:
import pandas as pd
import numpy as np
import torch
import requests

from bs4 import BeautifulSoup
from transformers import AutoTokenizer,AutoModelForSequenceClassification
import re

In [87]:
#df = pd.read_csv('../data/raw/Reviews.csv')

In [88]:
# Instantiate the Model
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [89]:
# Encode and Calculte Sentiment
tokens = tokenizer.encode('', return_tensors='pt')
result = model(tokens)

In [90]:
int(torch.argmax(result.logits)) + 1

4

In [91]:
# Collect Reviews
pages = [20, 30, 40, 50, 60, 70, 80]
reviews = []

for page in pages:
    # Construct the URL with the 'start' parameter
    url = f'https://www.yelp.com/biz/mejico-sydney-2?start={page}'

    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    regex = re.compile('.*comment.*')
    results = soup.find_all('p', {'class': regex})

    # Append reviews from the current page to the 'reviews' list
    reviews.extend([result.text for result in results])



In [92]:
# load reviews into dataframe and score it
df = pd.DataFrame(reviews, columns=['review'])

def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [94]:
df['sentiment'] = df['review'].apply( lambda x: sentiment_score(x[:512]))

In [95]:
df

,review,sentiment
0,Ordered feed me for $59 along with that.. Food...,2
1,The most fun I've had at a restaurant in a lon...,5
2,The restaurant itself is hip and modern. Expo...,3
3,I'd love to give them a 5 star but for the pri...,3
4,Though not a traditional Mexican Restaurant- t...,3
...,...,...
62,Funky new mexican restaurant serving up a good...,4
63,"Overrated and expensive. As others have said, ...",2
64,"Cool vibe, but that's about it. Guacamole com...",1
65,Overrated and expensive for what is served. Fo...,1


In [96]:
df['review'].iloc[0]

'Ordered feed me for $59 along with that.. Food was good quality but the service was tragic and wait was poorly managed. Personally if your with a group and looking for light finger food size portion then this would maybe be suitable for you,'

In [98]:
df['sentiment'].mean()

3.1492537313432836